In [ ]:
# Primer

The tutorial below is intended to provide quick illustrations of some of PsyNeuLink's basic and more advanced
capabilities. They assume some experience with computational modeling and/or relevant background knowledge.

*Installation and Setup:*

In [ ]:
%%capture
%pip install psyneulink

In [ ]:
import psyneulink as pnl
import numpy as np

## Simple Configurations

Mechanisms can be executed on their own (to gain familiarity with their operation, or for use in other Python
applications), or linked together and run in a Composition to implement part of, or an entire model. Linking
Mechanisms for execution can be as simple as creating them and then assigning them to a Composition in a list --
PsyNeuLink provides the necessary Projections that connect each to the next one in the list, making reasonable
assumptions about their connectivity. The following example creates a 3-layered 5-2-5 feedforward neural network,
the first layer of which takes an array of length 5 as its input, and uses a `Linear` function
(the default for a `ProcessingMechanism`), and the other two of which take 1d arrays of the specified sizes and use a
`Logistic` function:



In [ ]:
# Construct the Mechanisms:
input_layer = pnl.ProcessingMechanism(input_shapes=5, name="Input") # We can name the Mechanisms to make them easier to identify. If we don't specify certain parameters, PsyNeuLink will use default values for them. For example, the default function for a ProcessingMechanism is Linear, so we don't need to specify it here.
hidden_layer = pnl.ProcessingMechanism(input_shapes=2, function=pnl.Logistic, name='Hidden')
output_layer = pnl.ProcessingMechanism(input_shapes=5, function=pnl.Logistic, name='Output')

# Construct the Composition:
my_network = pnl.Composition(pathways=[[input_layer, hidden_layer, output_layer]]) # Note that pathways expects a list of lists

Each of the Mechanisms can be executed individually, by simply calling its `execute` method
with an appropriately-sized input array, for example:

In [ ]:
output_layer.execute([0, 2.5, 10.9, 2, 7.6])

The Composition connects the Mechanisms into a pathway that form a graph, which can be shown using its show_graph method:



In [ ]:
my_network.show_graph(output_fmt='jupyter') # The output_fmt argument specifies the format of the graph output; here we use 'jupyter' to display it inline in a Jupyter notebook.

Note that the Input Mechanism for the Composition is colored green (to designate it is an `INPUT` node), and its output Mechanism is colored Red (to designate it at a `OUTPUT` node).

As the name of the ``show_graph()`` method suggests, Compositions are represented in PsyNeuLink as graphs, using a
standard dependency dictionary format, so that they can also be submitted to other graph theoretic packages for
display and/or analysis (such as [NetworkX](https://networkx.github.io) and [igraph](http://igraph.org/redirect.html).  They can also be exported as a JSON file, in a format that is currently being developed for the exchange
of computational models in neuroscience and psychology (see `mdf` (ModECI) module in this documentation for more details).

In [ ]:
my_network.run([1, 4.7, 3.2, 6, 2])

The order in which Mechanisms appear in the list of the **pathways** argument of the Composition's constructor
determines their order in the pathway.

More complicated arrangements can be created by adding nodes individually using a Composition's `add_nodes` method, and/or by creating intersecting pathways, as shown in some of the examples further below.

PsyNeuLink picks sensible defaults when necessary Components are not specified. In the example above no `Projections` were specified, so PsyNeuLink automatically created the appropriate types (in this case, `MappingProjections`), and sized them appropriately to connect each pair of Mechanisms. Each Projection has a `matrix` parameter that weights the connections between the elements of the
output of its `sender` and those of the input to its `receiver`.  Here, the default is to use a fully connected matrix (the keyword for this in PsyNeuLink is `FULL_CONNECTIVITY_MATRIX`, keywords will be explained later), that connects every element of the sender's array to every element of the receiver's array with a weight of 1. However, it is easy to specify a Projection explicitly, including its matrix, simply by inserting them in between the Mechanisms in the pathway. Here, we also show how to add a pathway with the method `add_linear_processing_pathway` instead of `pathways` argument of the Composition's constructor:

**ATTENTION**:
Here, we also recreate the mechanisms. Not doing is a common source of confusion for new users and can result in unexpected behavior. As a rule of thumb, it is best to create all the components of a Composition before adding them and not reusing Mechanisms that were already added to a Composition.

In [ ]:
# Construct the Mechanisms:
input_layer = pnl.ProcessingMechanism(input_shapes=5, name="Input")
hidden_layer = pnl.ProcessingMechanism(input_shapes=2, function=pnl.Logistic, name='Hidden')
output_layer = pnl.ProcessingMechanism(input_shapes=5, function=pnl.Logistic, name='Output')

# Construct the Projection:
my_projection = pnl.MappingProjection(matrix=(.2 * np.random.rand(5, 2)) - .1)

# Creating the Composition and adding the pathway:
my_network = pnl.Composition()
my_network.add_linear_processing_pathway(pathway=[input_layer, my_projection, hidden_layer, output_layer])

my_network.run([1, 4.7, 3.2, 6, 2])

The first line above creates a Projection with a 2x5 matrix of random weights constrained to be between -.1 and +.1,
which is then inserted in the pathway between the ``input_layer`` and ``hidden_layer``. PsyNeuLink also alows to add the matrix itself instead of first creating a Projection, as shown below:

In [ ]:
# Construct the Mechanisms:
input_layer = pnl.ProcessingMechanism(input_shapes=5, name="Input")
hidden_layer = pnl.ProcessingMechanism(input_shapes=2, function=pnl.Logistic, name='Hidden')
output_layer = pnl.ProcessingMechanism(input_shapes=5, function=pnl.Logistic, name='Output')

# Creating the Composition and adding the pathway:
my_network = pnl.Composition()
my_network.add_linear_processing_pathway(pathway=[input_layer, (.2 * np.random.rand(5, 2)) - .1, hidden_layer, output_layer])

my_network.run([1, 4.7, 3.2, 6, 2])

PsyNeuLink is also flexible. For example, a recurrent Projection from the ``output_layer`` back to the ``hidden_layer`` can be added simply by adding another entry to the pathway:

In [ ]:
my_network.add_linear_processing_pathway([output_layer, hidden_layer])

*Note*, PsyNeuLink does not duplicate or replace pathways. The line of code is equivalent to the following:
```python
my_network.add_linear_processing_pathway(pathway=[input_layer, hidden_layer, output_layer, output_layer, hidden_layer])
```

We can also explicitly add projections without defining a pathway:

In [ ]:
input_layer = pnl.ProcessingMechanism(input_shapes=5, name="Input")
hidden_layer = pnl.ProcessingMechanism(input_shapes=2, function=pnl.Logistic, name='Hidden')
output_layer = pnl.ProcessingMechanism(input_shapes=5, function=pnl.Logistic, name='Output')

my_network = pnl.Composition()
my_network.add_linear_processing_pathway([input_layer, hidden_layer, output_layer])
recurrent_projection = pnl.MappingProjection(sender=output_layer,
                      receiver=hidden_layer)
my_network.add_projection(projection=recurrent_projection)

Let's inspect the Composition's graph again to see the new Projection:

In [ ]:
my_network.show_graph(output_fmt='jupyter')

The compositions now has two output nodes which are colored red. Let's run the Composition to see how the recurrence affects the output. To make it easier to see what is going on, we change the hidden and output layer function to `Linear` and set the layer size to 2 for each layer. This means that activation is just passed through layers without any transformation:

In [ ]:
input_layer = pnl.ProcessingMechanism(input_shapes=2, name="Input")
hidden_layer = pnl.ProcessingMechanism(input_shapes=2, name='Hidden')
output_layer = pnl.ProcessingMechanism(input_shapes=2, name='Output')

my_network = pnl.Composition()
my_network.add_linear_processing_pathway([input_layer, hidden_layer, output_layer])
recurrent_projection = pnl.MappingProjection(sender=output_layer,
                      receiver=hidden_layer)
my_network.add_projection(projection=recurrent_projection)

Run the composition with an input of `[1, 2]` (what do you expect the output to be?):

In [ ]:
my_network.run([1, 2])

The output is `[1, 2]` which is the output of the output_layer:

In [ ]:
hidden_layer.value

In [ ]:
input_layer.value

In [ ]:
output_layer.variable